In [124]:
#Where we Import 
from pyspark.sql.functions import col, lit, expr, when, to_timestamp
from pyspark.sql.types import *
from datetime import datetime
import time
import pyspark
from pyspark.sql import SparkSession
import random
import string
import pandas as pd
from pyspark.sql.functions import avg
import plotly.express as px
import pandas as pd
import numpy as np
import time

In [125]:
#ALEX Q1
##+===========================================================SET DATASETLENGTH HERE!!!
DataLength=1000000

In [126]:
#ALEX Q1

#Reading the original Raw city dataframes, found from the internet.
#They contain names of cities and population.
OG_F_path=r"C:\Users\alexw\Desktop\BDG3_FTP\OG_French_Cities.csv"
OG_G_path=r"C:\Users\alexw\Desktop\BDG3_FTP\OG_German_Cities.csv"
dfF=pd.read_csv(OG_F_path)
dfG=pd.read_csv(OG_G_path)
#Renaming Cities to City
dfF.rename(columns={'cities': 'city'}, inplace=True)

In [127]:
#ALEX Q1
#Checking the original French set out


dfF.head()


,city,population
0,Aast,1347387
1,Abainville,912312
2,Abancourt,870664
3,Abancourt,1666354
4,Abaucourt,839601


In [128]:
#Checking the original German set out:
dfG.head()


,city,population
0,Berlin,3644826.0
1,Hamburg,1841179.0
2,Munich,1471508.0
3,Cologne,1085664.0
4,Frankfurt,753056.0


In [129]:
#ALEX Q1
# # GRADER YOU WILL NEED TO UNCOMMENT THIS TO RUN CODE ON YOUR MACHINE!

# #GENERATING RANDOM CITY NAMES
# #French cities will end with 'ville'
# #German cities will end with 'burg'
# def generate_city_name(suffix):
#     return ''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyz'), np.random.randint(4, 7))) + suffix

# # This function makes random population within the range of existing df's population
# def generate_population():
#     return np.random.randint(0, 4000000)

# #I commented all this because it is unnecessary after the final dataset is made
# #It takes A LONG TIME to run

# # Adding French cities
# additional_data_F = pd.DataFrame({
#     'city': [generate_city_name('ville') for _ in range(DataLength)],
#     'population': [generate_population() for _ in range(DataLength)]
# })

# # Adding German cities
# additional_data_G = pd.DataFrame({
#     'city': [generate_city_name('burg') for _ in range(DataLength)],
#     'population': [generate_population() for _ in range(DataLength)]
# })

# # Concatenate the random data with the original data
# dfF_ext = pd.concat([dfF, additional_data_F], ignore_index=True)
# dfG_ext = pd.concat([dfG, additional_data_G], ignore_index=True)

In [131]:
#ALEX Q1
##!!GRADER YOU WILL NEED TO UNCOMMENT THIS TO RUN CODE ON YOUR MACHINE!
# ##This is generating the other columns.


# ##The areas are based on actual Km^2 data from the largest cities of the counties (Paris and Berlin)
# ##I just made up the cultural sites (:^|
# G_Areamin = 20
# G_Areamax = 900
# F_Areamax=110
# F_Areamin=20
# cultmin = 0
# cultmax = 100

# # Function to generate random CityID
# def generate_city_id():
#     return ''.join(str(np.random.randint(0, 10)) for _ in range(6))

# # Generate random values for dfF
# dfF_ext['Area'] = np.random.uniform(F_Areamin, F_Areamax, size=len(dfF_ext)).round(2)
# dfF_ext['Cultural Sites'] = np.random.randint(cultmin, cultmax, size=len(dfF_ext))
# dfF_ext['CityID'] = [generate_city_id() for _ in range(len(dfF_ext))]

# # Generate random values for dfG
# dfG_ext['Area'] = np.random.uniform(G_Areamin, G_Areamax, size=len(dfG_ext)).round(2)
# dfG_ext['Cultural Sites'] = np.random.randint(cultmin, cultmax, size=len(dfG_ext))
# dfG_ext['CityID'] = [generate_city_id() for _ in range(len(dfG_ext))]


In [135]:
#ALEX Q1
##!GRADER YOU WILL NEED TO UNCOMMENT THIS TO RUN CODE ON YOUR MACHINE!

# #Renaming some columns because I generated the data wrong
# #FRANCE
# dfF_ext.rename(columns={'city': 'CityName'}, inplace=True)
# dfF_ext.rename(columns={'population': 'Population'}, inplace=True)
# dfF_ext.rename(columns={'Cultural Sites':"CulturalSites"}, inplace=True)

# #Germany
# dfG_ext.rename(columns={'city': 'CityName'}, inplace=True)
# dfG_ext.rename(columns={'population': 'Population'}, inplace=True)
# dfG_ext.rename(columns={'Cultural Sites':"CulturalSites"}, inplace=True)

# #For some reasons the German population is double rather than integer so lets fix that
# dfG_ext['Population'].fillna(0, inplace=True)
# dfG_ext['Population'] = dfG_ext['Population'].astype(int)

In [137]:
#ALEX Q1
# # Save to CSV files: ===================== !GRADER YOU WILL NEED TO UNCOMMENT THIS TO RUN CODE ON YOUR MACHINE!

# dfG_ext.to_csv('german_cities.csv', index=False)
# dfF_ext.to_csv('french_cities.csv', index=False) 

In [138]:
#ALEX Q1
#Calling the paths to the saved Dataframes
frenchpath=r"french_cities.csv"
Germpath=r"german_cities.csv"

In [139]:
#Establishing A spark Session
spark = SparkSession.builder \
    .appName("Assignment 1") \
    .getOrCreate()

In [140]:
#ALEX Q1
#This will set and apply a schema for the French Cities dataset
F_schema = StructType([
    StructField("CityName", StringType(), nullable=False),
    StructField("Population", IntegerType(), nullable=False),
    StructField("Area", DoubleType(), nullable=False),
    StructField("CulturalSites", IntegerType(), nullable=False),
    StructField("CityID", IntegerType(), nullable=False),
])

french_cities = spark.read.format("csv").load(frenchpath, schema=F_schema)

In [141]:
#ALEX Q1
#This will set a schema for the German Cities Dataset
G_schema = StructType([
    StructField("CityName", StringType(), nullable=False),
    StructField("Population", IntegerType(), nullable=False),
    StructField("Area", DoubleType(), nullable=False),
    StructField("CulturalSites", IntegerType(), nullable=False),
    StructField("CityID", IntegerType(), nullable=False),
])


german_cities = spark.read.format("csv").load(Germpath, schema=G_schema)

In [146]:
#ALEX Q1
#Double checking that it all worked
#This is just to drop the first row of each dataframe which for some reason is just the column names and null values
german_cities=german_cities.dropna()
french_cities=french_cities.dropna()
#Show german_cities
german_cities.show()

+----------+----------+------+-------------+------+
|  CityName|Population|  Area|CulturalSites|CityID|
+----------+----------+------+-------------+------+
|    Berlin|   3644826|128.01|           80|741495|
|   Hamburg|   1841179|703.15|           65|839935|
|    Munich|   1471508|565.96|           72|923454|
|   Cologne|   1085664| 70.83|           65|443743|
| Frankfurt|    753056|468.15|           21|870108|
|    Bremen|    724909|312.68|           78|857508|
| Stuttgart|    634830|351.55|            3|660923|
|Düsseldorf|    619294|630.07|            0|357511|
|  Dortmund|    587010|241.18|           86|492983|
|     Essen|    583109|411.48|           29|434642|
|   Dresden|    555805|378.03|           12|743702|
|   Leipzig|    542529|257.83|           54|984606|
|  Hannover|    538068| 301.7|           50|929813|
| Nuremberg|    518365|853.29|           27|532105|
|  Duisburg|    498590|526.99|           40|237135|
|    Bochum|    364628|726.35|           85| 35023|
| Wuppertal|

In [144]:
#ALEX Q2
#Pyspark Dataframe Operation
#Here we are selecting both data frames and ordering them by area. 
#We can see French cities are a bit larger by area

#set start time (paul)
q2_pyspark_start_time = time.time()

germ_city_sizes = german_cities.select("CityName", "Area", "Population").orderBy(col("Area").desc()).show()
french_city_sizes = french_cities.select("CityName", "Area", "Population").orderBy(col("Area").desc()).show()
#============================================

#stop timer (paul)
q2_pyspark_end_time = time.time()
#calculate query time (paul)
q2_pyspark_time = q2_pyspark_end_time - q2_pyspark_start_time
print(f"PySpark execution time: {q2_pyspark_time} seconds")

+----------+------+----------+
|  CityName|  Area|Population|
+----------+------+----------+
| axfwzburg| 900.0|   1040561|
|  udslburg| 900.0|   3114441|
|tuaogwburg| 900.0|   3150189|
|pebqrtburg| 900.0|   3868220|
| vrxnnburg| 900.0|   3578065|
|  aqmjburg| 900.0|   1195098|
| jjarcburg| 900.0|   2787663|
|bcuycyburg|899.99|   3316963|
|vndgmoburg|899.99|   1889310|
|wuiiucburg|899.99|   2075381|
| tcaoyburg|899.99|   2065918|
|  bokwburg|899.99|   2013581|
|hlcxmzburg|899.99|   3599719|
| acrhyburg|899.99|   3670138|
|  xtusburg|899.99|   2230695|
| mndpqburg|899.99|   2313934|
| eogxhburg|899.99|   1880261|
|kmqcafburg|899.99|   1475948|
| uhfbmburg|899.99|    440347|
|  iojtburg|899.99|   1355627|
+----------+------+----------+
only showing top 20 rows

+-----------+-----+----------+
|   CityName| Area|Population|
+-----------+-----+----------+
| zwpoaville|110.0|    925330|
|  reksville|110.0|   2260056|
|efqiwhville|110.0|   3603321|
|  xqxgville|110.0|   3239647|
|  qpznville|

In [145]:
#Alex Q2
#SQL Version
#Making temp views so I can use SQL
german_cities.createOrReplaceTempView("german_cities")
french_cities.createOrReplaceTempView("french_cities")

#set start time (paul)
q2_sql_start_time = time.time()

# German City Call
germ_city_sizes = spark.sql("""
    
    SELECT CityName, Area, Population 
    FROM german_cities 
    ORDER BY Area DESC

""")
germ_city_sizes.show()

#French City Call
french_city_sizes = spark.sql("""
    
    SELECT CityName, Area, Population 
    FROM french_cities 
    ORDER BY Area DESC

""")

#stop timer (paul)
q2_sql_end_time = time.time()
#calculate query time (paul)
q2_sql_time = q2_sql_end_time - q2_sql_start_time
print(f"SQL execution time: {q2_sql_time} seconds")

# Show the result for French cities
french_city_sizes.show()

#Show difference in query execution times (paul)
print(f"Performance difference: PySpark was {q2_sql_time / q2_pyspark_time:.2f} times {'slower' if q2_sql_time > q2_pyspark_time else 'faster'} than SQL")

+----------+------+----------+
|  CityName|  Area|Population|
+----------+------+----------+
|pebqrtburg| 900.0|   3868220|
| jjarcburg| 900.0|   2787663|
|tuaogwburg| 900.0|   3150189|
|  aqmjburg| 900.0|   1195098|
| axfwzburg| 900.0|   1040561|
| vrxnnburg| 900.0|   3578065|
|  udslburg| 900.0|   3114441|
| uhfbmburg|899.99|    440347|
|bcuycyburg|899.99|   3316963|
|wuiiucburg|899.99|   2075381|
| mndpqburg|899.99|   2313934|
|kmqcafburg|899.99|   1475948|
|  bokwburg|899.99|   2013581|
| acrhyburg|899.99|   3670138|
|ajuasfburg|899.99|   3448173|
|vndgmoburg|899.99|   1889310|
|  xtusburg|899.99|   2230695|
|hlcxmzburg|899.99|   3599719|
| tcaoyburg|899.99|   2065918|
| eogxhburg|899.99|   1880261|
+----------+------+----------+
only showing top 20 rows

+-----------+-----+----------+
|   CityName| Area|Population|
+-----------+-----+----------+
|gbodrdville|110.0|   2650171|
|  fbfdville|110.0|   1324384|
|   Chalampé|110.0|     61912|
| lrjhcville|110.0|   2289232|
|tedrssville|

In [ ]:
#q3 paul

from pyspark.sql.functions import sum, avg, min, max, count

#pyspark solutions
print("PySpark Solutions:")

#set start time (paul)
q3_pyspark_start_time = time.time()

#aggregate for german cities
germany_agg = german_cities.agg(sum("Population").alias("TotalPopulation"),
                                 avg("Area").alias("AverageArea"),
                                 min("Population").alias("MinPopulation"),
                                 max("Population").alias("MaxPopulation"),
                                 count("CityID").alias("CityCount"))
germany_agg.show()

#aggregate for french cities
france_agg = french_cities.agg(sum("Population").alias("TotalPopulation"),
                               avg("Area").alias("AverageArea"),
                               min("Population").alias("MinPopulation"),
                               max("Population").alias("MaxPopulation"),
                               count("CityID").alias("CityCount"))
france_agg.show()

#stop timer (paul)
q3_pyspark_end_time = time.time()
#calculate query time (paul)
q3_pyspark_time = q3_pyspark_end_time - q3_pyspark_start_time
print(f"PySpark execution time: {q3_pyspark_time} seconds")

#sql solutions
print("\nSQL Solutions:")

#set start time (paul)
q3_sql_start_time = time.time()

#aggregate for german cities
germany_agg_sql = spark.sql("""
    SELECT SUM(Population) AS TotalPopulation,
           AVG(Area) AS AverageArea,
           MIN(Population) AS MinPopulation,
           MAX(Population) AS MaxPopulation,
           COUNT(CityID) AS CityCount
    FROM german_cities
""")
germany_agg_sql.show()

#aggregate for french cities
france_agg_sql = spark.sql("""
    SELECT SUM(Population) AS TotalPopulation,
           AVG(Area) AS AverageArea,
           MIN(Population) AS MinPopulation,
           MAX(Population) AS MaxPopulation,
           COUNT(CityID) AS CityCount
    FROM french_cities
""")

#stop timer (paul)
q3_sql_end_time = time.time()
#calculate query time (paul)
q3_sql_time = q3_sql_end_time - q3_sql_start_time
print(f"SQL execution time: {q3_sql_time} seconds")

france_agg_sql.show()

#Show difference in query execution times (paul)
print(f"Performance difference: PySpark was {q3_sql_time / q3_pyspark_time:.2f} times {'slower' if q3_sql_time > q3_pyspark_time else 'faster'} than SQL")

In [15]:
#Q5 Charan

#set start time (paul)
q5_pyspark_start_time = time.time()

full_outer_join_df = german_cities.join(french_cities, "CityID", "full_outer")\
                                   .select(german_cities["CityName"].alias("GermanCityName"),
                                           french_cities["CityName"].alias("FrenchCityName"),
                                           german_cities["Population"].alias("PopulationInGermany"),
                                           french_cities["Population"].alias("PopulationInFrance"))

# Show the combined DataFrame
full_outer_join_df.show()

#stop timer (paul)
q5_pyspark_end_time = time.time()
#calculate query time (paul)
q5_pyspark_time = q5_pyspark_end_time - q5_pyspark_start_time
print(f"PySpark execution time: {q5_pyspark_time} seconds")


+--------------+--------------+-------------------+------------------+
|GermanCityName|FrenchCityName|PopulationInGermany|PopulationInFrance|
+--------------+--------------+-------------------+------------------+
|         yPGeN|          Arrm|            3898854|           1533884|
|         BqIqF|        YpXVTA|            2626349|           1677669|
|        oPMhti|         iCIMy|            2258897|           1116640|
|         Gqdcx|         Ppsnh|            4242141|            634336|
|         tIutr|          DEsQ|             446469|           2698268|
|         kYnhs|        bbpzqc|            4848747|           1554811|
|        cVYwaO|         QxElg|            4181455|           1303426|
|          CjDj|          Xqjr|            4353703|           4622090|
|         ogzIT|        wHzKXB|             216939|            671899|
|         HFOnE|          VneL|            1345457|           4164036|
|          Lszg|          qvmt|            2713054|           1789120|
|     

In [16]:
#Q5 SQL

#set start time (paul)
q5_sql_start_time = time.time()

# SQL Query to perform full outer join 
query = """
SELECT 
    g.CityName AS GermanCityName,
    f.CityName AS FrenchCityName,
    g.Population AS PopulationInGermany,
    f.Population AS PopulationInFrance
FROM german_cities g
FULL OUTER JOIN french_cities f ON g.CityID = f.CityID
"""

# Execute the SQL query using Spark SQL
full_outer_join_df = spark.sql(query)

# Show the results of the SQL query
full_outer_join_df.show()

#stop timer (paul)
q5_sql_end_time = time.time()
#calculate query time (paul)
q5_sql_time = q5_sql_end_time - q5_sql_start_time
print(f"SQL execution time: {q5_sql_time} seconds")

#Show difference in query execution times (paul)
print(f"Performance difference: PySpark was {q5_sql_time / q5_pyspark_time:.2f} times {'slower' if q5_sql_time > q5_pyspark_time else 'faster'} than SQL")

+--------------+--------------+-------------------+------------------+
|GermanCityName|FrenchCityName|PopulationInGermany|PopulationInFrance|
+--------------+--------------+-------------------+------------------+
|         yPGeN|          Arrm|            3898854|           1533884|
|         BqIqF|        YpXVTA|            2626349|           1677669|
|        oPMhti|         iCIMy|            2258897|           1116640|
|         Gqdcx|         Ppsnh|            4242141|            634336|
|         tIutr|          DEsQ|             446469|           2698268|
|         kYnhs|        bbpzqc|            4848747|           1554811|
|        cVYwaO|         QxElg|            4181455|           1303426|
|          CjDj|          Xqjr|            4353703|           4622090|
|         ogzIT|        wHzKXB|             216939|            671899|
|         HFOnE|          VneL|            1345457|           4164036|
|          Lszg|          qvmt|            2713054|           1789120|
|     

In [17]:
#ALEX Q6
#Pyspark Dataframe operation

#set start time (paul)
q6_pyspark_start_time = time.time()

#For this question I am using
print("Germany")
german_cities.select(
    avg("CityID").alias("Avg_CityID"),
    avg("Population").alias("Avg_Population"),
    avg("Area").alias("Avg_Area"),
    avg("CulturalSites").alias("Avg_CulturalSites")
).show()

print("France")
french_cities.select(
    avg("CityID").alias("Avg_CityID"),
    avg("Population").alias("Avg_Population"),
    avg("Area").alias("Avg_Area"),
    avg("CulturalSites").alias("Avg_CulturalSites")
).show()


#As we can see they are similar sizes, with Germany having slightly more cultural sites and population
#This makes sense that both would be similar as they are randomly generated via the same parameters

#stop timer (paul)
q6_pyspark_end_time = time.time()
#calculate query time (paul)
q6_pyspark_time = q6_pyspark_end_time - q6_pyspark_start_time
print(f"PySpark execution time: {q6_pyspark_time} seconds")


Germany
+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499948.9589232053|2585.7041581891945|4.499701501492493|
+----------+------------------+------------------+-----------------+

France
+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499667.3588442057|2584.0123058284817| 4.50078249608752|
+----------+------------------+------------------+-----------------+



In [18]:
#Alex Q6
#SQL Version

#set start time (paul)
q6_sql_start_time = time.time()

#Using the previously created views in Q2
#German query
avg_german_stats = spark.sql("""
    
    SELECT AVG(CityID) AS Avg_CityID,
           AVG(Population) AS Avg_Population,
           AVG(Area) AS Avg_Area,
           AVG(CulturalSites) AS Avg_CulturalSites
    FROM german_cities
    
""")

avg_german_stats.show()

# French Query
avg_french_stats = spark.sql("""

    SELECT AVG(CityID) AS Avg_CityID,
           AVG(Population) AS Avg_Population,
           AVG(Area) AS Avg_Area,
           AVG(CulturalSites) AS Avg_CulturalSites
    FROM french_cities
    
""")
avg_french_stats.show()

#stop timer (paul)
q6_sql_end_time = time.time()
#calculate query time (paul)
q6_sql_time = q6_sql_end_time - q6_sql_start_time
print(f"SQL execution time: {q6_sql_time} seconds")

#Show difference in query execution times (paul)
print(f"Performance difference: PySpark was {q6_sql_time / q6_pyspark_time:.2f} times {'slower' if q6_sql_time > q6_pyspark_time else 'faster'} than SQL")

+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499948.9589232053|2585.7041581891945|4.499701501492493|
+----------+------------------+------------------+-----------------+

+----------+------------------+------------------+-----------------+
|Avg_CityID|    Avg_Population|          Avg_Area|Avg_CulturalSites|
+----------+------------------+------------------+-----------------+
|  500003.0|2499667.3588442057|2584.0123058284817| 4.50078249608752|
+----------+------------------+------------------+-----------------+



In [19]:
#Q4 Srinivas

#set start time (paul)
q4_pyspark_start_time = time.time()

germany_density = german_cities.withColumn("Density", col("Population") / col("Area"))
france_density = french_cities.withColumn("Density", col("Population") / col("Area"))
germany_density.select("CityName", "Density").show()
france_density.select("CityName", "Density").show()

#stop timer (paul)
q4_pyspark_end_time = time.time()
#calculate query time (paul)
q4_pyspark_time = q4_pyspark_end_time - q4_pyspark_start_time
print(f"PySpark execution time: {q4_pyspark_time} seconds")


+---------+------------------+
| CityName|           Density|
+---------+------------------+
|   Berlin| 3949.287557500281|
|  Hamburg|2370.2330508474574|
|   Munich| 4666.881235918893|
|  Cologne|2616.3149450820683|
|Frankfurt|            1466.0|
|     tfOB| 498.2321165493534|
|     uBMt|3756.3574614760746|
|     fpbw|104.04242996776689|
|     rOXV| 547.7717676827241|
|    dApbQ| 562.0234644967117|
|   zIXZbo|1049.9719393890805|
|   WoiTtZ|1091.6647137835444|
|     AtMM| 787.1854868654482|
|    EfLQd|16940.895207419337|
|   iKbycv| 499.0313445086107|
|     bQQH|  2379.42325951114|
|     DYiw|1077.3815788611744|
|    qzQcV| 4364.230797090202|
|    XozDq| 866.3320888244964|
|   KvnMpM|24.474152204574725|
+---------+------------------+
only showing top 20 rows

+---------+------------------+
| CityName|           Density|
+---------+------------------+
|    Paris|            9000.0|
|Marseille|3536.9908561928514|
|     Lyon|10256.945895132652|
| Toulouse| 3778.529163144548|
|     Nice| 4

In [20]:
#set start time (paul)
q4_sql_start_time = time.time()

# Germany Density Calculation
print("Germany")
density_german_stats = spark.sql("""
SELECT CityName, Population / Area AS Density
FROM german_cities

""")

density_german_stats.show()

# French Density Calculation
print("France")

density_french_stats = spark.sql("""
SELECT CityName, Population / Area AS Density
FROM french_cities

""")

density_french_stats.show()

#stop timer (paul)
q4_sql_end_time = time.time()
#calculate query time (paul)
q4_sql_time = q4_sql_end_time - q4_sql_start_time
print(f"SQL execution time: {q4_sql_time} seconds")

#Show difference in query execution times (paul)
print(f"Performance difference: PySpark was {q4_sql_time / q4_pyspark_time:.2f} times {'slower' if q4_sql_time > q4_pyspark_time else 'faster'} than SQL")



Germany
+---------+------------------+
| CityName|           Density|
+---------+------------------+
|   Berlin| 3949.287557500281|
|  Hamburg|2370.2330508474574|
|   Munich| 4666.881235918893|
|  Cologne|2616.3149450820683|
|Frankfurt|            1466.0|
|     tfOB| 498.2321165493534|
|     uBMt|3756.3574614760746|
|     fpbw|104.04242996776689|
|     rOXV| 547.7717676827241|
|    dApbQ| 562.0234644967117|
|   zIXZbo|1049.9719393890805|
|   WoiTtZ|1091.6647137835444|
|     AtMM| 787.1854868654482|
|    EfLQd|16940.895207419337|
|   iKbycv| 499.0313445086107|
|     bQQH|  2379.42325951114|
|     DYiw|1077.3815788611744|
|    qzQcV| 4364.230797090202|
|    XozDq| 866.3320888244964|
|   KvnMpM|24.474152204574725|
+---------+------------------+
only showing top 20 rows

France
+---------+------------------+
| CityName|           Density|
+---------+------------------+
|    Paris|            9000.0|
|Marseille|3536.9908561928514|
|     Lyon|10256.945895132652|
| Toulouse| 3778.529163144548

In [21]:

top_20_cities = french_cities.orderBy("Population", ascending=False).limit(20)
city_names = top_20_cities.select("CityName").collect()
cultural_sites = top_20_cities.select("CulturalSites").collect()
city_names = [row.CityName for row in city_names]
cultural_sites = [row.CulturalSites for row in cultural_sites]


data_dict = {'CityName': city_names, 'CulturalSites': cultural_sites}
df = pd.DataFrame(data_dict)
fig = px.bar(df, x="CityName", y="CulturalSites",
             title="Cultural Sites in Top 20 Populous French Cities")
fig.update_layout(xaxis_title="City", yaxis_title="Number of Cultural Sites")
fig.show()



In [22]:
top_20_cities_german = german_cities.orderBy("Population", ascending=False).limit(20)
city_names = top_20_cities_german.select("CityName").collect()
cultural_sites = top_20_cities_german.select("CulturalSites").collect()
city_names = [row.CityName for row in city_names]
cultural_sites = [row.CulturalSites for row in cultural_sites]
data_dict = {'CityName': city_names, 'CulturalSites': cultural_sites}
df = pd.DataFrame(data_dict)
fig = px.bar(df, x="CityName", y="CulturalSites",
             title="Cultural Sites in Top 20 Populous German Cities")
fig.update_layout(xaxis_title="City", yaxis_title="Number of Cultural Sites")
fig.show()